# In this notebook, we divide the data into trimester interval. We use preprocessed content and lda_model from LDA_Content.ipynb

In [1]:
import pickle
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [2]:
lda_list=[]
content_list=[]
for i in range(0,7):    
    lda_list.append('lda_model/lda_model_{}.sav'.format(i))
    content_list.append('content/content_preprocessed-{}.sav'.format(i))

In [3]:
lda=[]
content=[]

for i in range(0,7):
    content.append(pickle.load(open(content_list[i], 'rb')).tolist())
    k=pickle.load(open(lda_list[i], 'rb')).print_topics()
    temp=[]
    for x in range(0, 6):
        temp.append(pickle.load(open(lda_list[i], 'rb')).print_topics()[x][1])
    lda.append(temp)

c:\users\fazam\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
hash_list=[]
for i in range(0,7):
    subhash=[]
    for j in range(0,6):
        ss=lda[i][j].split(" + ")
        hash_map={}
        for k in ss:
            sss = k.split('*')
            hash_map[sss[1].replace('"','')]=float(sss[0])
        subhash.append(hash_map)
    hash_list.append(subhash)

In [5]:
hash_list

[[{'april': 0.017,
   'custom': 0.016,
   'munoz': 0.027,
   'news': 0.018,
   'offer': 0.024,
   'report': 0.015,
   'tell': 0.015,
   'travel': 0.017,
   'tuesday': 0.017,
   'week': 0.018},
  {'associ': 0.019,
   'chang': 0.026,
   'crew': 0.02,
   'custom': 0.017,
   'express': 0.019,
   'issu': 0.022,
   'munoz': 0.03,
   'polici': 0.019,
   'press': 0.02,
   'respons': 0.018},
  {'carrier': 0.034,
   'fli': 0.018,
   'go': 0.019,
   'hour': 0.017,
   'overbook': 0.017,
   'plan': 0.021,
   'time': 0.019,
   'viral': 0.015,
   'week': 0.019,
   'year': 0.025},
  {'allow': 0.029,
   'board': 0.028,
   'chang': 0.019,
   'employe': 0.018,
   'gate': 0.019,
   'peopl': 0.02,
   'polici': 0.021,
   'ticket': 0.023,
   'travel': 0.036,
   'twitter': 0.021},
  {'aircraft': 0.019,
   'airport': 0.022,
   'aviat': 0.017,
   'employe': 0.018,
   'leav': 0.016,
   'louisvill': 0.016,
   'overbook': 0.017,
   'post': 0.019,
   'report': 0.017,
   'secur': 0.019},
  {'aviat': 0.025,
   'depar

In [6]:
def scoring(hashlist, wordlist):
    each_result= 0
    for x in wordlist:
        if x not in list(hashlist.keys()):
            continue
        else:
            each_result += hashlist[x]
    return each_result

In [8]:
article = pd.read_json("articles.json")
article= article.sort_values(by="publication_timestamp")
article['created_at']=article['publication_timestamp'].apply(lambda x: dt.fromtimestamp(x/1000)).dt.strftime('%Y-%m')
datetime = article['created_at'].drop_duplicates().tolist()

## dx contain classified topic for each article. We compare each word in content with each word in lda_model to generate score and classified by the highest score

In [9]:
trimestering={}
for i in datetime:
    if datetime.index(i)==28:
        trimestering[i] = int((datetime.index(i)-1)/4)
    else:
        trimestering[i]= int(datetime.index(i)/4)
article['trimester']=article['created_at'].map(trimestering)
dx = []
for i in range(0,7):
    x = article[article["trimester"]==i][["article_id","created_at"]].reset_index(drop=True)
    dx.append(x)

In [10]:
dx[0].head()

,article_id,created_at
0,76509020-d093-11e6-8d6f-9f4c7b0f94de,2017-01
1,b5c0ce50-d0a2-11e6-ab89-c11942898bd6,2017-01
2,502a51e0-d0a9-11e6-ab89-c11942898bd6,2017-01
3,61489c80-d0b7-11e6-919c-7789120ed8c4,2017-01
4,2ad85e20-d0c0-11e6-ab89-c11942898bd6,2017-01


In [12]:
for i in range(0,7):
    for j in range(0,6):
        for k in range(0, dx[i].shape[0]):
            dx[i].at[k, "Topic_{}_Score".format(j)]= scoring(hash_list[i][j], content[i][k])

In [13]:
dx[0].Topic_0_Score.value_counts()

0.000    1385
0.017     501
0.015     355
0.033     326
0.018     301
         ... 
0.292       1
0.346       1
0.283       1
0.674       1
0.439       1
Name: Topic_0_Score, Length: 1298, dtype: int64

In [14]:
dx[0].head()

,article_id,created_at,Topic_0_Score,Topic_1_Score,Topic_2_Score,Topic_3_Score,Topic_4_Score,Topic_5_Score
0,76509020-d093-11e6-8d6f-9f4c7b0f94de,2017-01,0.000,0.060,0.105,0.028,0.038,0.036
1,b5c0ce50-d0a2-11e6-ab89-c11942898bd6,2017-01,0.000,0.040,0.130,0.028,0.038,0.052
2,502a51e0-d0a9-11e6-ab89-c11942898bd6,2017-01,0.065,0.054,0.094,0.021,0.000,0.088
3,61489c80-d0b7-11e6-919c-7789120ed8c4,2017-01,0.015,0.080,0.168,0.069,0.073,0.146
4,2ad85e20-d0c0-11e6-ab89-c11942898bd6,2017-01,0.000,0.020,0.050,0.000,0.000,0.016


In [15]:
for i in range(0,7):
    for j in range(0, dx[i].shape[0]):
        hcounter=0
        scounter=0
        for counter in range(1,6):
            if dx[i].loc[j, "Topic_{}_Score".format(counter)]>dx[i].loc[j, "Topic_{}_Score".format(hcounter)]:
                scounter=hcounter
                hcounter=counter
            elif dx[i].loc[j, "Topic_{}_Score".format(counter)]>dx[i].loc[j, "Topic_{}_Score".format(scounter)]:
                scounter=counter
            elif scounter==hcounter:
                scounter=counter
        dx[i].at[j, "Which_Topic"]= int(hcounter)
        if dx[i].loc[j, "Topic_{}_Score".format(hcounter)]>1.25* dx[i].loc[j, "Topic_{}_Score".format(scounter)]:
            dx[i].at[j, "Unclear_Topic"]= "No"
        else:
            dx[i].at[j, "Unclear_Topic"]= "Yes, with Topic {}".format(scounter)
            

In [16]:
dx[0].head()

,article_id,created_at,Topic_0_Score,Topic_1_Score,Topic_2_Score,Topic_3_Score,Topic_4_Score,Topic_5_Score,Which_Topic,Unclear_Topic
0,76509020-d093-11e6-8d6f-9f4c7b0f94de,2017-01,0.000,0.060,0.105,0.028,0.038,0.036,2.0,No
1,b5c0ce50-d0a2-11e6-ab89-c11942898bd6,2017-01,0.000,0.040,0.130,0.028,0.038,0.052,2.0,No
2,502a51e0-d0a9-11e6-ab89-c11942898bd6,2017-01,0.065,0.054,0.094,0.021,0.000,0.088,2.0,"Yes, with Topic 5"
3,61489c80-d0b7-11e6-919c-7789120ed8c4,2017-01,0.015,0.080,0.168,0.069,0.073,0.146,2.0,"Yes, with Topic 5"
4,2ad85e20-d0c0-11e6-ab89-c11942898bd6,2017-01,0.000,0.020,0.050,0.000,0.000,0.016,2.0,No


In [17]:
dx[0]["Which_Topic"].value_counts()

5.0    3806
1.0    2880
3.0    2731
2.0    2538
0.0    2434
4.0    1621
Name: Which_Topic, dtype: int64

In [18]:
dx[0]["Unclear_Topic"].value_counts()

No                   10175
Yes, with Topic 0     1391
Yes, with Topic 1     1162
Yes, with Topic 5      945
Yes, with Topic 4      909
Yes, with Topic 3      812
Yes, with Topic 2      616
Name: Unclear_Topic, dtype: int64

In [19]:
dx[0][dx[0]["Unclear_Topic"]=="Yes, with Topic 0"]

,article_id,created_at,Topic_0_Score,Topic_1_Score,Topic_2_Score,Topic_3_Score,Topic_4_Score,Topic_5_Score,Which_Topic,Unclear_Topic
43,0648ed40-d16a-11e6-ab89-c11942898bd6,2017-01,0.063,0.060,0.044,0.020,0.072,0.056,4.0,"Yes, with Topic 0"
125,71e3f550-d5c4-11e6-a9e9-8b2016dfe8ad,2017-01,0.041,0.000,0.044,0.036,0.000,0.000,2.0,"Yes, with Topic 0"
130,10003c60-d2c5-11e6-ab89-c11942898bd6,2017-01,0.081,0.020,0.038,0.089,0.000,0.000,3.0,"Yes, with Topic 0"
147,927d87e0-d43e-11e6-ad25-9bd934315e4c,2017-01,0.040,0.043,0.000,0.019,0.000,0.000,1.0,"Yes, with Topic 0"
165,b86880a0-d582-11e6-ab89-c11942898bd6,2017-01,0.224,0.000,0.265,0.036,0.070,0.000,2.0,"Yes, with Topic 0"
...,...,...,...,...,...,...,...,...,...,...
15900,ae143b20-2c40-11e7-b065-c11942898bd6,2017-04,0.266,0.276,0.165,0.212,0.089,0.227,1.0,"Yes, with Topic 0"
15954,1ab2dd30-2c8d-11e7-b065-c11942898bd6,2017-04,0.102,0.060,0.019,0.000,0.000,0.104,5.0,"Yes, with Topic 0"
15956,e0bd1e60-2c91-11e7-b4a8-65792358983c,2017-04,0.180,0.207,0.069,0.019,0.105,0.119,1.0,"Yes, with Topic 0"
15965,2ca68d80-2ccd-11e7-aba5-d37f0d6d833b,2017-04,0.018,0.000,0.019,0.000,0.000,0.000,2.0,"Yes, with Topic 0"


In [20]:
dx[0]

,article_id,created_at,Topic_0_Score,Topic_1_Score,Topic_2_Score,Topic_3_Score,Topic_4_Score,Topic_5_Score,Which_Topic,Unclear_Topic
0,76509020-d093-11e6-8d6f-9f4c7b0f94de,2017-01,0.000,0.060,0.105,0.028,0.038,0.036,2.0,No
1,b5c0ce50-d0a2-11e6-ab89-c11942898bd6,2017-01,0.000,0.040,0.130,0.028,0.038,0.052,2.0,No
2,502a51e0-d0a9-11e6-ab89-c11942898bd6,2017-01,0.065,0.054,0.094,0.021,0.000,0.088,2.0,"Yes, with Topic 5"
3,61489c80-d0b7-11e6-919c-7789120ed8c4,2017-01,0.015,0.080,0.168,0.069,0.073,0.146,2.0,"Yes, with Topic 5"
4,2ad85e20-d0c0-11e6-ab89-c11942898bd6,2017-01,0.000,0.020,0.050,0.000,0.000,0.016,2.0,No
...,...,...,...,...,...,...,...,...,...,...
16005,342ef120-2d98-11e7-b065-c11942898bd6,2017-04,0.030,0.020,0.108,0.000,0.193,0.033,4.0,No
16006,48bdcb10-2d9e-11e7-b065-c11942898bd6,2017-04,0.231,0.274,0.301,0.535,0.304,0.272,3.0,No
16007,93799f30-2da3-11e7-b4a8-65792358983c,2017-04,0.175,0.248,0.143,0.186,0.201,0.227,1.0,"Yes, with Topic 5"
16008,5be3baa0-2dae-11e7-b065-c11942898bd6,2017-04,0.166,0.083,0.211,0.282,0.251,0.058,3.0,"Yes, with Topic 4"


In [21]:
dx[1]

,article_id,created_at,Topic_0_Score,Topic_1_Score,Topic_2_Score,Topic_3_Score,Topic_4_Score,Topic_5_Score,Which_Topic,Unclear_Topic
0,3e956a80-2dcb-11e7-aba5-d37f0d6d833b,2017-05,0.154,0.340,0.827,0.087,0.875,0.040,4.0,"Yes, with Topic 2"
1,9e7b0bf0-2dc9-11e7-b065-c11942898bd6,2017-05,0.071,0.088,0.016,0.089,0.177,0.119,4.0,No
2,400e03f0-2dcf-11e7-b065-c11942898bd6,2017-05,0.000,0.000,0.000,0.019,0.000,0.000,3.0,No
3,aaeee8a0-2e98-11e7-9360-9bd934315e4c,2017-05,0.000,0.034,0.075,0.019,0.049,0.000,2.0,No
4,fbbcc340-2dd7-11e7-b065-c11942898bd6,2017-05,0.038,0.028,0.016,0.019,0.037,0.022,0.0,"Yes, with Topic 4"
...,...,...,...,...,...,...,...,...,...,...
4862,51d9ace0-8dd5-11e7-83f4-ebb7ea9adb61,2017-08,0.073,0.036,0.192,0.054,0.273,0.475,5.0,No
4863,5c612620-8de4-11e7-83f4-ebb7ea9adb61,2017-08,0.016,0.080,0.325,0.137,0.223,0.497,5.0,No
4864,eae6c690-8e50-11e7-83f4-ebb7ea9adb61,2017-08,0.034,0.017,0.048,0.034,0.071,0.066,4.0,"Yes, with Topic 5"
4865,1e0c48f0-8e57-11e7-83f4-ebb7ea9adb61,2017-08,0.000,0.021,0.126,0.322,0.380,0.093,4.0,"Yes, with Topic 3"
